# 向 satpy 添加自定义读取器

为了向 satpy 添加一个读取器，你需要创建两个文件：- 用于描述要读取的文件和可用数据集的YAML文件- 一个实现数据集和元数据实际读取的Python文件
在本教程中，我们将为SEVIRI数据实现一个Eumetsat NetCDF格式的读取器。

## YAML 文件
YAML文件由三个部分组成：- `reader` 部分，为 reader 提供基本参数- `file_types` 部分，该部分给出此读取器可处理的文件类型- `datasets` 部分，描述此读取器可用的数据集
### `reader` 部分`reader` 部分，为 reader 提供基本参数。
在此部分中需要提供的参数为：- 描述: 读卡器的一般描述- name: 这是读取器的名称，应与文件名相同（显然不包括.yaml扩展名）。这是在satpy中交互使用时的名称，因此请慎重选择！一种松散的约定是使用`<format>_<instrument>_<level>`作为名称的模板- 传感器: 此读取器支持的传感器列表- reader: 要使用的元阅读器，在大多数情况下，`FileYAMLReader` 是一个不错的选择。

```yaml
reader:
  description: NetCDF4 reader for the Eumetsat MSG format
  name: nc_seviri_l1b
  sensors: [seviri]
  reader: !!python/name:satpy.readers.yaml_reader.FileYAMLReader
```

### `file_types` 部分
每种文件类型需要提供：- `file_reader`，用于此读取器的类，该类将处理文件，您将在对应的Python文件中实现该类（见下一部分）- `file_patterns`，用于匹配此读取器可处理的文件的模式。语法基本上与 `format` 相同，但增加了时间部分。有关详细信息，请参阅 [trollsift 包文档](https://trollsift.readthedocs.io/en/latest/usage.html)。- 可选地，文件类型可以包含一个 `requires` 字段：它是一个列表，列出当前文件类型正常运行所需的其他文件类型。例如，HRIT MSG 格式的段文件每个都需要一个前导文件和尾随文件才能正确读取，因此在这种情况下，我们在文件类型定义中添加了 `requires: [HRIT_PRO, HRIT_EPI]`。 


```yaml
file_types:
    nc_seviri_l1b:
        file_reader: !!python/name:satpy.readers.nc_seviri_l1b.NCSEVIRIFileHandler
        file_patterns: ['W_XX-EUMETSAT-Darmstadt,VIS+IR+IMAGERY,{satid:4s}+SEVIRI_C_EUMG_{processing_time:%Y%m%d%H%M%S}.nc']
    nc_seviri_l1b_hrv:
        file_reader: !!python/name:satpy.readers.nc_seviri_l1b.NCSEVIRIHRVFileHandler
        file_patterns: ['W_XX-EUMETSAT-Darmstadt,HRV+IMAGERY,{satid:4s}+SEVIRI_C_EUMG_{processing_time:%Y%m%d%H%M%S}.nc']
```

### `数据集` 部分
数据集部分描述了文件中可用的每个数据集。提供的参数可供实现类的方法使用。
您可以定义的参数示例包括：- 名称- 传感器- 分辨率- 波长- 极化- 标准名称：用于数据集的名称，用于识别数据类型并正确处理它- 单位: 数据的单位，确保在多个平台/仪器上进行一致的处理- 修饰符：已对数据应用的修改，例如 `sunz_corrected`- 文件类型- 坐标：这指示需要加载哪些数据集以操作当前数据集- 以及任何对读者相关的其他字段
本部分可以简单地从现有的SEVIRI读取器（例如`msg_native`读取器）复制和适配。

```yaml

datasets:
  HRV:
    name: HRV
    resolution: 1000.134348869
    wavelength: [0.5, 0.7, 0.9]
    calibration:
      reflectance:
        standard_name: toa_bidirectional_reflectance
        units: "%"
      radiance:
        standard_name: toa_outgoing_radiance_per_unit_wavelength
        units: W m-2 um-1 sr-1
      counts:
        standard_name: counts
        units: count
    file_type: nc_seviri_l1b_hrv

  IR_016:
    name: IR_016
    resolution: 3000.403165817
    wavelength: [1.5, 1.64, 1.78]
    calibration:
      reflectance:
        standard_name: toa_bidirectional_reflectance
        units: "%"
      radiance:
        standard_name: toa_outgoing_radiance_per_unit_wavelength
        units: W m-2 um-1 sr-1
      counts:
        standard_name: counts
        units: count
    file_type: nc_seviri_l1b
    nc_key: 'ch3'

  IR_039:
    name: IR_039
    resolution: 3000.403165817
    wavelength: [3.48, 3.92, 4.36]
    calibration:
      brightness_temperature:
        standard_name: toa_brightness_temperature
        units: K
      radiance:
        standard_name: toa_outgoing_radiance_per_unit_wavelength
        units: W m-2 um-1 sr-1
      counts:
        standard_name: counts
        units: count
    file_type: nc_seviri_l1b
    nc_key: 'ch4'

  IR_087:
    name: IR_087
    resolution: 3000.403165817
    wavelength: [8.3, 8.7, 9.1]
    calibration:
      brightness_temperature:
        standard_name: toa_brightness_temperature
        units: K
      radiance:
        standard_name: toa_outgoing_radiance_per_unit_wavelength
        units: W m-2 um-1 sr-1
      counts:
        standard_name: counts
        units: count
    file_type: nc_seviri_l1b

  IR_097:
    name: IR_097
    resolution: 3000.403165817
    wavelength: [9.38, 9.66, 9.94]
    calibration:
      brightness_temperature:
        standard_name: toa_brightness_temperature
        units: K
      radiance:
        standard_name: toa_outgoing_radiance_per_unit_wavelength
        units: W m-2 um-1 sr-1
      counts:
        standard_name: counts
        units: count
    file_type: nc_seviri_l1b

  IR_108:
    name: IR_108
    resolution: 3000.403165817
    wavelength: [9.8, 10.8, 11.8]
    calibration:
      brightness_temperature:
        standard_name: toa_brightness_temperature
        units: K
      radiance:
        standard_name: toa_outgoing_radiance_per_unit_wavelength
        units: W m-2 um-1 sr-1
      counts:
        standard_name: counts
        units: count
    file_type: nc_seviri_l1b

  IR_120:
    name: IR_120
    resolution: 3000.403165817
    wavelength: [11.0, 12.0, 13.0]
    calibration:
      brightness_temperature:
        standard_name: toa_brightness_temperature
        units: K
      radiance:
        standard_name: toa_outgoing_radiance_per_unit_wavelength
        units: W m-2 um-1 sr-1
      counts:
        standard_name: counts
        units: count
    file_type: nc_seviri_l1b

  IR_134:
    name: IR_134
    resolution: 3000.403165817
    wavelength: [12.4, 13.4, 14.4]
    calibration:
      brightness_temperature:
        standard_name: toa_brightness_temperature
        units: K
      radiance:
        standard_name: toa_outgoing_radiance_per_unit_wavelength
        units: W m-2 um-1 sr-1
      counts:
        standard_name: counts
        units: count
    file_type: nc_seviri_l1b

  VIS006:
    name: VIS006
    resolution: 3000.403165817
    wavelength: [0.56, 0.635, 0.71]
    calibration:
      reflectance:
        standard_name: toa_bidirectional_reflectance
        units: "%"
      radiance:
        standard_name: toa_outgoing_radiance_per_unit_wavelength
        units: W m-2 um-1 sr-1
      counts:
        standard_name: counts
        units: count
    file_type: nc_seviri_l1b

  VIS008:
    name: VIS008
    resolution: 3000.403165817
    wavelength: [0.74, 0.81, 0.88]
    calibration:
      reflectance:
        standard_name: toa_bidirectional_reflectance
        units: "%"
      radiance:
        standard_name: toa_outgoing_radiance_per_unit_wavelength
        units: W m-2 um-1 sr-1
      counts:
        standard_name: counts
        units: count
    file_type: nc_seviri_l1b

  WV_062:
    name: WV_062
    resolution: 3000.403165817
    wavelength: [5.35, 6.25, 7.15]
    calibration:
      brightness_temperature:
        standard_name: toa_brightness_temperature
        units: "K"
      radiance:
        standard_name: toa_outgoing_radiance_per_unit_wavelength
        units: W m-2 um-1 sr-1
      counts:
        standard_name: counts
        units: count
    file_type: nc_seviri_l1b

  WV_073:
    name: WV_073
    resolution: 3000.403165817
    wavelength: [6.85, 7.35, 7.85]
    calibration:
      brightness_temperature:
        standard_name: toa_brightness_temperature
        units: "K"
      radiance:
        standard_name: toa_outgoing_radiance_per_unit_wavelength
        units: W m-2 um-1 sr-1
      counts:
        standard_name: counts
        units: count
    file_type: nc_seviri_l1b

    
```

YAML文件现已准备就绪，让我们继续处理对应的Python文件。## Python 文件Python文件需要为我们要读取的每种文件类型实现一个文件处理器类。- 此类需要实现一些方法：- the `__init__` 方法，该方法接受参数- 文件名 (字符串)- 通过使用 YAML 文件中定义的模式解析文件名所获得的文件名信息 (dict)- 从YAML文件中的文件类型定义中获取的文件类型信息  
此方法在当前文件类型有特定要求时，也可以接收其他文件处理实例作为参数。（请参见上方YAML文件中关于filetype部分的说明）- `get_dataset` 方法，该方法接受以下参数- 要加载的数据集的ID- 数据集信息，即YAML文件中通道的描述  
此方法在加载成功时必须返回一个包含加载数据集的数据和元数据的 xarray.DataArray 实例，或在加载不成功时返回 None。- `get_area_def` 方法，该方法接受单个参数，即我们想要获取区域的数据集ID。对于无法通过区域定义进行地理定位的数据，需要从 `get_dataset` 加载像素坐标，以便对生成的场景进行导航。也就是说，如果数据无法通过区域定义进行地理定位，则数据集部分应指定 `coordinates: [longitude_dataset, latitude_dataset]`。- 可选地，如果对于此数据类型需要按区域筛选文件，可以实现 `get_bounding_box` 方法
除此之外，需要定义两个属性：`start_time` 和 `end_time`，用于定义感知的开始和结束时间。

In [ ]:
# this is nc_seviri_l1b.py
class NCSEVIRIFileHandler():
    def __init__(self, filename, filename_info, filetype_info):
        super(NCSEVIRIFileHandler, self).__init__(filename, filename_info, filetype_info)
        self.nc = None
    
    def get_dataset(self, dataset_id, dataset_info):
        if dataset_id.calibration != 'radiance':
            # TODO: implement calibration to relfectance or brightness temperature
            return
        if self.nc is None:
            self.nc = xr.open_dataset(self.filename,
                                      decode_cf=True,
                                      mask_and_scale=True,
                                      chunks={'num_columns_vis_ir': CHUNK_SIZE,
                                              'num_rows_vis_ir': CHUNK_SIZE})
            self.nc = self.nc.rename({'num_columns_vir_ir': 'x', 'num_rows_vir_ir': 'y'})
        dataset = self.nc[dataset_info['nc_key']]
        dataset.attrs.update(dataset_info)
        return dataset
    
    def get_area_def(self, dataset_id):
        # TODO
        pass
    
class NCSEVIRIHRVFileHandler():
  # left as an exercise to the reader :)